In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"

s3_bucket = "s3://mist-secorapp-{env}/rrm-local/rrm-local-{env}/".format(env=env)

date_day = "2020-03-0*"
hr = '*'

rrm_local_path = "dt={day}/hr={hr}/*.seq".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)



user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-secorapp-production/rrm-local/rrm-local-production/dt=2020-03-0*/hr=*/*.seq


In [2]:
rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)


In [7]:
rdd_rrm_local.take(2)

[(58978958,
  bytearray(b'{"updated": true, "clients": 1, "pre_bandwidth": 20, "power": 12, "timestamp": 1583020800, "site_id": "f2543ad9-331f-48de-984f-4f1190837df1", "org_id": "95c2ff76-5b0a-47b4-a162-de9c4d3c46ba", "band": "24", "bandwidth": 20, "rad_id": "r24", "rrm_bandwidth": 20, "rrm_power": 12, "rrm_channel": 6, "pre_power": 12, "when": "2020-03-01T00:00:00", "rrm_enabled": true, "reason": "non-wifi", "ap_id": "5c5b350e2b57", "source_data": "random.no_rrm_channel", "command": "capacity-anomaly", "channel_update_interval": 7200, "pre_channel": 1, "channel": 6}')),
 (58978959,
  bytearray(b'{"updated": true, "clients": 1, "pre_bandwidth": 20, "power": 14, "timestamp": 1583020800, "site_id": "2f8d3161-8562-48dd-8630-737190cc06b3", "org_id": "bbb101eb-b62d-4fb1-8c3d-030c6db7e208", "band": "24", "bandwidth": 20, "rad_id": "r24", "rrm_bandwidth": 20, "rrm_power": 14, "rrm_channel": 6, "pre_power": 14, "when": "2020-03-01T00:00:00", "rrm_enabled": true, "reason": "non-wifi", "ap_id": 

In [5]:
df_rrm = rdd_rrm_local.map(lambda x: json.loads(x[1])).toDF()
df_rrm.show()

+------------+----+---------+-------+-----------------------+-------+--------------------+--------------------+-----+-------------+-----------+---------+------+--------------+-------------+-----------+-----------+---------+--------------------+--------------------+----------+-------+-------------------+
|       ap_id|band|bandwidth|channel|channel_update_interval|clients|             command|              org_id|power|pre_bandwidth|pre_channel|pre_power|rad_id|        reason|rrm_bandwidth|rrm_channel|rrm_enabled|rrm_power|             site_id|         source_data| timestamp|updated|               when|
+------------+----+---------+-------+-----------------------+-------+--------------------+--------------------+-----+-------------+-----------+---------+------+--------------+-------------+-----------+-----------+---------+--------------------+--------------------+----------+-------+-------------------+
|5c5b350e2b57|  24|       20|      6|                   7200|      1|    capacity-ano

In [6]:
df_rrm.printSchema()

root
 |-- ap_id: string (nullable = true)
 |-- band: string (nullable = true)
 |-- bandwidth: long (nullable = true)
 |-- channel: long (nullable = true)
 |-- channel_update_interval: long (nullable = true)
 |-- clients: long (nullable = true)
 |-- command: string (nullable = true)
 |-- org_id: string (nullable = true)
 |-- power: long (nullable = true)
 |-- pre_bandwidth: long (nullable = true)
 |-- pre_channel: long (nullable = true)
 |-- pre_power: long (nullable = true)
 |-- rad_id: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- rrm_bandwidth: long (nullable = true)
 |-- rrm_channel: long (nullable = true)
 |-- rrm_enabled: boolean (nullable = true)
 |-- rrm_power: long (nullable = true)
 |-- site_id: string (nullable = true)
 |-- source_data: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- updated: boolean (nullable = true)
 |-- when: string (nullable = true)



In [ ]:
df_USNYELE = df_rrm.filter((col('site_id')==user_site_USNYELE) & (col('band')==5))
df_USFLLON = df_rrm.filter((col('site_id')==user_site_USFLLON) & (col('band')==5))

In [ ]:
df_USNYELE.select("command", "reason").groupBy("command", 'reason').count().show()

In [ ]:
df_USFLLON.select("command", "reason").groupBy("command", 'reason').count().show()

In [ ]:
df_USFLLON.filter(col("command")=="rrm-global").groupBy("band",'when', "command", 'reason').count().show()